In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
import numpy as np
from os.path import basename
from typing import Dict, Callable

from pathlib import Path
import pickle

import pandas as pd

from collections import namedtuple
from tqdm import tqdm_notebook
from operator import itemgetter

%matplotlib inline

sys.path.append('/home/laugustyniak/github/phd/sentiment-backend/')
from aspects.analysis.nlp_architect import get_metrics, get_models_params_from_name, filter_datasets
from aspects.analysis.statistics_dataset import get_unique_words_from_corpus

/home/laugustyniak/anaconda3/envs/python36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/laugustyniak/anaconda3/envs/python36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
models_to_skip = ['char-bilstm', 'char-lstm']

In [3]:
def skip_char_models(model_name):
    return not any(m in model_name.as_posix() for m in models_to_skip)

In [4]:
list(Path('../models-2/glove.840B.300d/').glob('*'))

[PosixPath('../models-2/glove.840B.300d/model-info-char-bilstm-10epochs-Laptops_poria-train.conll.info'),
 PosixPath('../models-2/glove.840B.300d/model-info-word-lstm-crf-10epochs-Restaurants_poria-train.conll.info'),
 PosixPath('../models-2/glove.840B.300d/model-info-word-bilstm-crf-10epochs-Laptops_poria-train.conll.info'),
 PosixPath('../models-2/glove.840B.300d/model-info-char-lstm-10epochs-Restaurants_poria-train.conll.info'),
 PosixPath('../models-2/glove.840B.300d/model-info-word-bilstm-10epochs-Restaurants_poria-train.conll.info'),
 PosixPath('../models-2/glove.840B.300d/model-info-word-lstm-crf-10epochs-Laptops_poria-train.conll.info'),
 PosixPath('../models-2/glove.840B.300d/model-info-char-bilstm-crf-10epochs-Laptops_poria-train.conll.info'),
 PosixPath('../models-2/glove.840B.300d/model-info-char-word-lstm-crf-10epochs-Restaurants_poria-train.conll.info'),
 PosixPath('../models-2/glove.840B.300d/model-info-char-word-bilstm-10epochs-Laptops_poria-train.conll.info'),
 PosixPa

In [5]:
reindex_results_order = ['word bilstm', 'char word bilstm', 'word bilstm crf', 'char word bilstm crf', 'word lstm', 'char word lstm', 'word lstm crf', 'char word lstm crf']

In [6]:
model_f1_by_word_embedding = {}
for word_embedding_models_path in list(Path('../models-2/').glob('*')):
    models_f1 = {}
    
    models_paths = filter(skip_char_models, word_embedding_models_path.glob('*10epoch*'))
    models_metrics = get_metrics(models_paths)
    for model_name, model_metrics in models_metrics.items():
        if model_name not in models_to_skip:
            model_name = get_models_params_from_name(model_name)
            models_f1[model_name] = model_metrics.f1 
    if models_f1:
        model_f1_by_word_embedding[word_embedding_models_path.stem] = models_f1
    
df = pd.DataFrame.from_dict(model_f1_by_word_embedding).round(2).reindex(reindex_results_order)
print(df.to_latex())
df

\begin{tabular}{lrrrrr}
\toprule
{} &  crawl-300d-2M &  sentic2vec &  numberbatch-en &  glove.6B &  glove.840B \\
\midrule
word bilstm          &          70.31 &       58.66 &           58.34 &     55.52 &       75.18 \\
char word bilstm     &          73.71 &       66.28 &           62.28 &     62.83 &       67.80 \\
word bilstm crf      &          76.46 &       61.48 &           67.85 &     57.09 &       75.00 \\
char word bilstm crf &          76.94 &       68.13 &           68.28 &     63.85 &       76.31 \\
word lstm            &          68.01 &       57.70 &           53.74 &     47.87 &       69.72 \\
char word lstm       &          64.77 &       60.88 &           53.80 &     51.33 &       67.39 \\
word lstm crf        &          75.49 &       63.31 &           65.57 &     47.30 &       75.17 \\
char word lstm crf   &          74.37 &       65.14 &           66.73 &     59.81 &       73.83 \\
\bottomrule
\end{tabular}



,crawl-300d-2M,sentic2vec,numberbatch-en,glove.6B,glove.840B
word bilstm,70.31,58.66,58.34,55.52,75.18
char word bilstm,73.71,66.28,62.28,62.83,67.80
word bilstm crf,76.46,61.48,67.85,57.09,75.00
char word bilstm crf,76.94,68.13,68.28,63.85,76.31
word lstm,68.01,57.70,53.74,47.87,69.72
char word lstm,64.77,60.88,53.80,51.33,67.39
word lstm crf,75.49,63.31,65.57,47.30,75.17
char word lstm crf,74.37,65.14,66.73,59.81,73.83


In [7]:
# df.to_excel('/home/laugustyniak/luk.augustyniak@gmail.com/Projects/PRELUDIUM/Artykuły/aspect-based sentiment 2018/laptops-aspects.xlsx')

In [8]:
# def draw_metrics(models_metrics: Dict, title=None):
#     pd.DataFrame.from_dict({
#         get_models_params_from_name(k): v
#         for k, v
#         in list(models_metrics.items())
#     }, orient='index').sort_index(axis=0).plot(kind='bar', figsize=(25,12), title=title)

In [9]:
# draw_metrics(models_metrics)

In [10]:
# list(models_metrics.items())

# draw_metrics(dict(models_f1))

In [11]:
# for we_name, we_f1 in model_f1_by_word_embedding.items():
#     draw_metrics(dict(we_f1), title=we_name)

In [12]:
for word_embedding_models_path in list(Path('../models').glob('*')):
    models_f1 = []
    
    models_paths = word_embedding_models_path.glob('*10epoch*')

# Get WE word count

In [13]:
corpus_words = get_unique_words_from_corpus()

Corpus iterator: 49475it [00:00, 898239.95it/s]
Corpus iterator: 53781it [00:00, 986896.14it/s]
Corpus iterator: 12470it [00:00, 1008580.56it/s]
Corpus iterator: 13257it [00:00, 563562.44it/s]


In [14]:
corpus_words

{'Restaurants_poria-train': {'attended',
  'thougt',
  'dhosas',
  'effort',
  'cucumber',
  'others',
  'except',
  'taste',
  'yes',
  'untrendy',
  'ruby',
  'iced',
  'genuine',
  'sounds',
  'rest',
  'brunch',
  'lay',
  'undercooked-the',
  'exprience',
  '3',
  'sweat',
  'batter',
  'soo',
  'sit',
  'naan',
  'felt',
  'worked',
  'l',
  'distract',
  'pork',
  'deceiving',
  'counters',
  'showcase',
  'cry',
  'arigato',
  'highly',
  'potatoes',
  'exemplifies',
  'manhatten',
  '#',
  'include',
  'cant',
  'prepared',
  'brings',
  'waving',
  "frisco's",
  'rices',
  'glass',
  "katz's",
  'details',
  'u',
  'curry',
  'period',
  'long-time',
  'occasion',
  'friends',
  'informative',
  'self',
  'two',
  'vegetarian',
  'around',
  'borderline',
  'toasting',
  'mentioned',
  'pre-theatre',
  'reminds',
  'weekend',
  'year',
  'volare',
  'individuals',
  'especially',
  "lot's",
  'flight',
  'knows',
  'unwanted',
  'seeming',
  'chosing',
  'wong',
  'frat',
  '

In [15]:
def load_word_embeddings(file_path):
    with open(file_path, encoding='utf-8') as fp:
        words = []
        try:
            for line in tqdm_notebook(fp, desc=file_path + ': embedding loading'):
                line_fields = line.split()
                if len(line_fields) < 5:
                    continue
                else:
                    if line[0] == ' ':
                        pass
                    else:
                        word = line_fields[0]
                        try:
                            pass
                        except:
                            continue
                        words.append(word)
        except UnicodeDecodeError:
            pass
    return set(words)

In [19]:
EMBEDDINGS_PATH = Path('/home/laugustyniak/data/embeddings/')

EMBEDDINGS = [
    'glove.6B.50d.txt',
    'glove.twitter.27B.200d.txt',
    'glove.840B.300d.txt',
    'numberbatch-en.txt',
    'crawl-300d-2M.vec',
    'sota-google.txt',
    'sota-retrofit-600.txt',
    'sota-sswe-50.txt',
    'sota-wiki-600.txt',
    'sentic2vec.txt',
]

In [34]:
def get_word_embeddings_vocabularies(word_embedding_names):
    return {
        word_embedding_name: load_word_embeddings((EMBEDDINGS_PATH / word_embedding_name).as_posix())
        for word_embedding_name 
        in tqdm_notebook(word_embedding_names)
    }
   
word_embedding_vocabulalaries = get_word_embeddings_vocabularies(EMBEDDINGS)

In [47]:
def get_datasets_coverage_with_word_embedding(corpus_words, word_embedding_vocabulalaries):
    word_embedding_coverage = {}
    for word_embedding_name, vocabulary in word_embedding_vocabulalaries.items():
        word_embedding_coverage[word_embedding_name] = {
            corpus_name: len(words.difference(vocabulary))/len(words)
            for corpus_name, words
            in corpus_words.items()
        }
    return word_embedding_coverage

In [48]:
{k: len(v) for k, v in  corpus_words.items()}

{'Restaurants_poria-train': 4619,
 'Laptops_poria-train': 4238,
 'Laptops_poria-test': 1922,
 'Restaurants_poria-test': 2270}

In [58]:
len(corpus_words['Restaurants_poria-test'] - corpus_words['Restaurants_poria-train'])

747

In [59]:
len(corpus_words['Laptops_poria-test'] - corpus_words['Laptops_poria-train'])

497

In [49]:
{word_embedding_name: len(vocabulary) for word_embedding_name, vocabulary in word_embedding_vocabulalaries.items()}

{'glove.6B.50d.txt': 400000,
 'glove.twitter.27B.200d.txt': 1193515,
 'glove.840B.300d.txt': 2195884,
 'numberbatch-en.txt': 417194,
 'crawl-300d-2M.vec': 1999995,
 'sota-google.txt': 60349,
 'sota-retrofit-600.txt': 29444,
 'sota-sswe-50.txt': 137052,
 'sota-wiki-600.txt': 29444,
 'sentic2vec.txt': 42007}

In [50]:
datasets_coverage_with_word_embedding = get_datasets_coverage_with_word_embedding(corpus_words, word_embedding_vocabulalaries)

In [56]:
lack_of_words_percentage = pd.DataFrame(datasets_coverage_with_word_embedding).round(4)*100

In [57]:
lack_of_words_percentage

,glove.6B.50d.txt,glove.twitter.27B.200d.txt,glove.840B.300d.txt,numberbatch-en.txt,crawl-300d-2M.vec,sota-google.txt,sota-retrofit-600.txt,sota-sswe-50.txt,sota-wiki-600.txt,sentic2vec.txt
Laptops_poria-test,4.27,8.17,2.24,9.83,2.39,10.51,12.54,6.14,12.54,17.17
Laptops_poria-train,9.18,11.70,4.20,14.06,4.32,18.92,20.86,11.99,20.86,18.55
Restaurants_poria-test,7.40,8.63,4.36,9.12,4.27,19.91,21.81,9.87,21.81,32.20
Restaurants_poria-train,10.78,11.99,6.11,13.70,5.98,26.65,28.38,14.09,28.38,34.38
